<a href="https://colab.research.google.com/github/look4pritam/ArtificialIntelligence/blob/master/ConvolutionalNeuralNetwork/Notebooks/TransferLearning/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification using Convolutional Neural Network

In this example, we will learn to classify images using [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network). 

We will use TensorFlow flower dataset for classification. 

See [link](https://www.tensorflow.org/datasets/catalog/tf_flowers) for more details on the dataset.

# Set the root directory for processing.

In [ ]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

# Import required python modules.

In [ ]:
import numpy as np
np.random.seed(7)

In [ ]:
import tensorflow as tf
tf.random.set_seed(7)

# Load Oxford flowers-102 dataset using TensorFlow dataset.

### Import required python modules.

In [ ]:
import tensorflow_datasets as tfds

### Define a dataset name and dataset splits.

In [ ]:
dataset_name = 'oxford_flowers102'
splits = ['test', 'validation', 'train']

### Download the dataset.

In [ ]:
dataset, dataset_info = tfds.load(dataset_name, split = splits, with_info=True, as_supervised=True)
(train_samples, validation_samples, test_samples) = dataset

### Show information about the dataset.

In [ ]:
print('Number of flower types - ', dataset_info.features['label'].num_classes)
print('Number of training examples - ', tf.data.experimental.cardinality(train_samples))
print('Number of validation examples - ', tf.data.experimental.cardinality(validation_samples))
print('Number of test examples - ', tf.data.experimental.cardinality(test_samples))

print('Flower types full list is as follows - ')
print(dataset_info.features['label'].names)

### Print the dataset information.

In [ ]:
print(dataset_info)

### Show dataset samples.

In [ ]:
tfds.show_examples(train_samples, dataset_info, rows=2, cols=8)

# Process the dataset.

### Define the image size and the model shape.

In [ ]:
image_size = 224
model_shape = (image_size, image_size, 3)

### Define a function to format samples.

In [ ]:
def format_sample(input_image, input_label):
    input_image = tf.cast(input_image, tf.float32)
    input_image = input_image / 255.0

    input_image = tf.image.resize(input_image, (image_size, image_size))

    return(input_image, input_label)

### Format train, validation, and test samples.

In [ ]:
train_dataset = train_samples.map(format_sample)
validation_dataset = validation_samples.map(format_sample)
test_dataset = test_samples.map(format_sample)

### Define a function to augment the train dataset.

In [ ]:
def augment_data(image, label):

  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_contrast(image, lower=0.0, upper=1.0)

  return image, label

### Augment the train dataset.

In [ ]:
train_dataset = train_dataset.map(augment_data)

### Define a buffer size and a batch size for processing the dataset.

In [ ]:
buffer_size = 1024
batch_size = 32

### Process the train datasset.

In [ ]:
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.repeat()

print(train_dataset)

### Process the validation dataset.

In [ ]:
validation_dataset = validation_dataset.batch(batch_size)
validation_dataset = validation_dataset.repeat()

print(validation_dataset)

### Process the test dataset.

In [ ]:
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

print(test_dataset)

# Define a CNN based classification model for training from scratch.

### Import required python modules.

In [ ]:
from tensorflow import keras

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D

### Use MobileNetV2 model for training.

In [ ]:
from tensorflow import keras

base_model = keras.applications.MobileNetV2(input_shape=model_shape, include_top=False, weights=None) #weights='imagenet')

### Show model summary.

In [ ]:
base_model.trainable = True
base_model.summary()

### Define model for image classification.

In [ ]:
def build_model():
    
    input_image = base_model.input

    x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    predictions = keras.layers.Dense(dataset_info.features['label'].num_classes, activation='softmax')(x)

    model = keras.Model(inputs=input_image, outputs=predictions)   

    return (model)

### Create image classification model and show the summary.

In [ ]:
model = build_model()
model.summary()

### Compile the model.



In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Define a callback function.

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

# Train the model.

### Define number of steps for train, validation, and test dataset.

In [ ]:
steps_per_epoch = round(tf.data.experimental.cardinality(train_samples).numpy())//batch_size
validation_steps = round(tf.data.experimental.cardinality(validation_samples).numpy())//batch_size
test_steps = round(tf.data.experimental.cardinality(test_samples).numpy())//batch_size

### Define number of epochs.

In [ ]:
epochs = 10

### Train the model.

In [ ]:
history = model.fit(train_dataset, 
                    steps_per_epoch=steps_per_epoch,
                    validation_data=validation_dataset, 
                    validation_steps=validation_steps,
                    batch_size=batch_size, 
                    epochs=epochs,
                    callbacks=[tensorboard_callback])

# Evaluate the trained model.

In [ ]:
evaluation_data = model.evaluate(test_dataset, steps=test_steps)
print('Test loss - {0:.4f}'.format(evaluation_data[0]))
print('Test accuracy - {0:.4f}'.format(evaluation_data[1]))

# Visualize training graphs.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs